In [81]:
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader

import os

# Đường dẫn thư mục chứa các file PDF
folder_path = "D:/vscode/Langchain/rag_langchain/data_source/vietnam"

# Lấy danh sách tất cả file PDF trong thư mục
pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]

# Load tất cả các file PDF
pdf_docs = []
for pdf_file in pdf_files:
    pdf_path = os.path.join(folder_path, pdf_file)  # Đường dẫn đầy đủ
    loader = PyPDFLoader(pdf_path)
    pdf_docs.extend(loader.load())  # Nối nội dung của từng file PDF vào danh sách


Ignoring wrong pointing object 0 0 (offset 0)


In [82]:
len(pdf_docs)

13

In [101]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 200
chunk_overlap = 100

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False,
)

docs = splitter.split_documents(pdf_docs)

In [102]:
len(docs)

206

In [85]:
docs

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': 'D:/vscode/Langchain/rag_langchain/data_source/vietnam\\Bài 10_ 85-92.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='85\nPHÂN TÍCH THỰC TRẠNG  \nTAI NẠN GIAO THÔNG ĐƯỜNG BỘ Ở VIỆT NAM\nThS. Hồ Trọng Du  \nTS. Nguyễn Văn Trạng \nKhoa Cơ khí Động lực, Trường đại học Sư phạm Kỹ thuật TP .HCM \nPGS.TS. Lý Hùng Anh \nKhoa Kỹ thuật Giao thông, Trường đại học Bách khoa TP .HCM\nTÓM TẮT:'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': 'D:/vscode/Langchain/rag_langchain/data_source/vietnam\\Bài 10_ 85-92.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='PGS.TS. Lý Hùng Anh \nKhoa Kỹ thuật Giao thông, Trường đại học Bách khoa TP .HCM\nTÓM TẮT:\nThực trạng tai nạn giao thông ở Việt Nam trong những năm gần đầy có nhiều tiến \ntriển tốt, số vụ tai nạn giao thông, số người chết và số người bị thương có xu hướng'),
 Document(

In [6]:
docs.pop(57)

Document(metadata={'source': 'https://arxiv.org/pdf/2312.16862', 'page': 4}, page_content='Submission and Formatting Instructions for the W ANT@ICML 2024\nLinear\nDown\nr\nLinear\nUpPretrained\nWeights\n\ud835\nInput\nOutput\nLoRA\n(a) LoRA\nMLP\nNormalization\nMHA LoRA\n(b) LoRA Module \nfor LLMs Block\nMHA\nQuery-Key \nNormalization\nLayer Norm\nRMS Norm\nMLP\nLoRA\nAfter \nStage1\n(c) LLMs Block\n for TinyGPT-V')

In [ ]:
docs

In [161]:
docss = docs[:57]

In [ ]:
docss

In [166]:
docsss = docs[58:190]

In [ ]:
docsss

In [103]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

model_name = "Cloyne/vietnamese-sbert-v3"
model = SentenceTransformer(model_name)
embedding_model = HuggingFaceEmbeddings(model_name = model_name)

In [9]:
text = "This is a sample query."

embedding = embedding_model.embed_query(text)
len(embedding)

768

1.Chrom vector database

In [104]:
from langchain_chroma import Chroma

chroma_db = Chroma.from_documents(documents=docs, embedding=embedding_model)

In [105]:
retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [97]:
retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8, "k": 5}  # Giảm threshold
)


In [106]:
query = "Số vụ tai nạn giao thông và hậu quả"

In [107]:
retriever.invoke(query)

[Document(id='86768dd4-6f11-40cb-be33-58a6eed42c4b', metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 7, 'page_label': '8', 'producer': 'PyPDF', 'source': 'D:/vscode/Langchain/rag_langchain/data_source/vietnam\\Bài 10_ 85-92.pdf', 'total_pages': 8}, page_content='tuoi-vang-lao-dong-la-tru-cot-gia-dinh-816824.vov,18/6/2023.\n[8] Tai nạn giao thông để lại hậu quả khủng khiếp và dai dẳng về kinh tế - xã hội.'),
 Document(id='73cbdacf-3e4f-4348-94a2-23f2e2339767', metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 1, 'page_label': '2', 'producer': 'PyPDF', 'source': 'D:/vscode/Langchain/rag_langchain/data_source/vietnam\\Bài 10_ 85-92.pdf', 'total_pages': 8}, page_content='nghiên cứu trước đây, 80% các vụ tai nạn giao thông xảy ra có nguyên nhân chính \nlà do người tham gia giao thông, do sự thiếu hiểu biết về pháp luật giao thông, quy \ntắc giao thông và văn hoá ứng xử khi tham gia giao thông [6]. Hậu quả tai nạn giao'),
 Document(id='b517a219-19b2-491c-adc5-69dee3e2adca',

In [59]:
similar_docs = chroma_db.similarity_search(query, k=4)

In [60]:
similar_docs

[Document(id='9203b90c-2811-4adc-a4fe-a60b83e995cb', metadata={'author': 'minh pham', 'creationdate': '2025-02-13T04:03:40+00:00', 'creator': 'Canva', 'keywords': 'DAGe9E8gS40,BAGe4X6zF2Q,0', 'moddate': '2025-02-13T04:03:40+00:00', 'page': 0, 'page_label': '1', 'producer': 'Canva', 'source': 'D:/vscode/Langchain/rag_langchain/data_source/vietnam/Trong năm 2024, tình hình vi phạm giao thông tại Việt Nam diễn biến phức tạp với các số liệu đáng chú ý như sau.pdf', 'title': 'Trong năm 2024, tình hình vi phạm giao thông tại Việt Nam diễn biến phức tạp với các số liệu đáng chú ý như sau:', 'total_pages': 2}, page_content='Ph ươ ng ti ệ n b ị  t ạ m gi ữ : Kho ả ng 1 tri ệ u ph ươ ng ti ệ n các lo ạ i.\nTrong đó, vi ph ạ m nồ ng đ ộ  cồ n chiếm 19,14% t ổ ng số vi ph ạ m, v ớ i 722.409 tr ườ ng h ợ p b ị  x ử  lý. \n3. Nguyên nhân chính dẫn đến tai n ạ n giao thông:'),
 Document(id='031cd91b-8792-4c4c-9609-787ecb963da9', metadata={'author': 'minh pham', 'creationdate': '2025-02-13T04:03

In [61]:
similar_docs = chroma_db.similarity_search(query, k=5)

# similar_docs sẽ chứa 4 tài liệu có độ tương đồng cao nhất với query
for doc in similar_docs:
    print(doc.page_content)

Ph ươ ng ti ệ n b ị  t ạ m gi ữ : Kho ả ng 1 tri ệ u ph ươ ng ti ệ n các lo ạ i.
Trong đó, vi ph ạ m nồ ng đ ộ  cồ n chiếm 19,14% t ổ ng số vi ph ạ m, v ớ i 722.409 tr ườ ng h ợ p b ị  x ử  lý. 
3. Nguyên nhân chính dẫn đến tai n ạ n giao thông:
S ử  d ụ ng r ượ u bia, chất kích thích có cồ n: Chiếm 4,89% số v ụ .
S ử  d ụ ng ma túy, chất gây nghi ệ n: Chiếm 0,18% số v ụ .
Vi ệ c không chấp hành các quy tắc giao thông, đ ặ c bi ệ t là vi ph ạ m nồ ng đ ộ  cồ n, là nguyên
1. Số v ụ  tai n ạ n giao thông và h ậ u qu ả :
T ổ ng số v ụ  tai n ạ n: 23.484 v ụ .
Số ng ườ i t ử  vong: 10.944 ng ườ i.
Số ng ườ i b ị  th ươ ng: 17.342 ng ườ i.
So v ớ i năm 2023, số v ụ  tai n ạ n tăng 0,7%, số ng ườ i chết gi ả m 7,7%, và số ng ườ i b ị  th ươ ng tăng 
4,2%.
T ổ ng số v ụ  tai n ạ n: 23.484 v ụ .
Số ng ườ i t ử  vong: 10.944 ng ườ i.
Số ng ườ i b ị  th ươ ng: 17.342 ng ườ i.
So v ớ i năm 2023, số v ụ  tai n ạ n tăng 0,7%, số ng ườ i chết gi ả m 7,7%, và số ng ườ 